# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [13]:
import numpy as np
import pandas as pd

def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    exp_solution = []
    for i, p in data.iterrows():
        data.loc[i, "Solution Range"] = data.loc[i, "Price in 1 Year"] / P - 1
        exp_solution.append(data.loc[i, "Probability"] * data.loc[i, "Solution Range"])
        answer = sum(exp_solution)
    return answer

# Test
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
print(exp_return(data,40))

0.07500000000000002


Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [17]:
import numpy as np
import pandas as pd

def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    exp_solution = []
    for i, p in data.iterrows():
        data.loc[i, "Solution Range"] = data.loc[i, "Price in 1 Year"] / P - 1
        exp_solution.append(data.loc[i, "Probability"] * (data.loc[i, "Solution Range"] - exp_return(data, P))**2)
    answer = np.sqrt(sum(exp_solution))
    return answer

# Test
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
print(volatility(data,40))

0.21301408404140795


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [22]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    return_exp = exp_return(data, P)
    vol_value = volatility(data, P)
    error = 1.96 * (vol_value / np.sqrt(len(data)))
    lower_bound = return_exp - error
    upper_bound = return_exp + error
    return [lower_bound, upper_bound]

# Test
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
print(interval(data,40))

[-0.16604812797447732, 0.3160481279744774]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [7]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return rf + beta * (market_return - rf)

# Test
print(exp_return_capm(-0.6,0.1, 0.02))

-0.028


### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [4]:
import pandas as pd

def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    # Read the CSV file into a DataFrame
    data = pd.read_csv(path)
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    data = data.dropna(subset=['RET'])
    permno_counts = data['PERMNO'].value_counts()
    valid_permnos = permno_counts[permno_counts >= 500].index
    cleaned_data = data[data['PERMNO'].isin(valid_permnos)]
    cleaned_data = cleaned_data[['PERMNO', 'date', 'CUSIP', 'RET']].reset_index(drop=True)
    return cleaned_data

# Test
print(clean_data("monthlycrsp_mod6.csv"))

        PERMNO        date     CUSIP       RET
0        10065  1970-01-30  00621210 -0.144375
1        10065  1970-02-27  00621210  0.105882
2        10065  1970-03-31  00621210  0.017857
3        10065  1970-04-30  00621210 -0.078947
4        10065  1970-05-29  00621210 -0.020952
...        ...         ...       ...       ...
403311   84129  2022-08-31  98970110  0.016315
403312   84129  2022-09-30  98970110 -0.075777
403313   84129  2022-10-31  98970110  0.021235
403314   84129  2022-11-30  98970110  0.005583
403315   84129  2022-12-30  98970110 -0.051332

[403316 rows x 4 columns]


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [25]:
import pandas as pd

def clean_data(path):
    data = pd.read_csv(path)
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    data = data.dropna(subset=['RET'])
    permno_counts = data['PERMNO'].value_counts()
    valid_permnos = permno_counts[permno_counts >= 500].index
    cleaned_data = data[data['PERMNO'].isin(valid_permnos)]
    cleaned_data = cleaned_data[['PERMNO', 'date', 'CUSIP', 'RET']].reset_index(drop=True)
    return cleaned_data

def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    avg_returns = data.groupby('CUSIP')['RET'].mean().reset_index()
    avg_returns.columns = ['CUSIP', 'Average Return']
    return avg_returns

#Test
print(avg_return(clean_data("monthlycrsp_mod6.csv")))

        CUSIP  Average Return
0    00036110        0.012326
1    00095710        0.011339
2    00105510        0.018398
3    00108M10        0.011101
4    00120410        0.012135
..        ...             ...
674  G6359F13        0.019803
675  G7665A10        0.013261
676  G8994E10        0.013812
677  H2717810        0.011593
678  N5946510        0.019437

[679 rows x 2 columns]


In [9]:
import pandas as pd

def clean_data(path):
    data = pd.read_csv(path)
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    data = data.dropna(subset=['RET'])
    permno_counts = data['PERMNO'].value_counts()
    valid_permnos = permno_counts[permno_counts >= 500].index
    cleaned_data = data[data['PERMNO'].isin(valid_permnos)]
    cleaned_data = cleaned_data[['PERMNO', 'date', 'CUSIP', 'RET']].reset_index(drop=True)
    return cleaned_data

def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    volatilities = data.groupby('CUSIP')['RET'].std().reset_index()
    volatilities.columns = ['CUSIP', 'Volatility']
    return volatilities

#Test
print(security_volatilities(clean_data("monthlycrsp_mod6.csv")))

        CUSIP  Volatility
0    00036110    0.122475
1    00095710    0.089881
2    00105510    0.099828
3    00108M10    0.171732
4    00120410    0.051050
..        ...         ...
674  G6359F13    0.205250
675  G7665A10    0.129473
676  G8994E10    0.090320
677  H2717810    0.131243
678  N5946510    0.134147

[679 rows x 2 columns]


Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [30]:
import pandas as pd

def clean_data(path):
    data = pd.read_csv(path)
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    data = data.dropna(subset=['RET'])
    permno_counts = data['PERMNO'].value_counts()
    valid_permnos = permno_counts[permno_counts >= 500].index
    cleaned_data = data[data['PERMNO'].isin(valid_permnos)]
    cleaned_data = cleaned_data[['PERMNO', 'date', 'CUSIP', 'RET']].reset_index(drop=True)
    return cleaned_data


def avg_return(data):
    avg_returns = data.groupby('CUSIP')['RET'].mean().reset_index()
    avg_returns.columns = ['CUSIP', 'Average Return']
    return avg_returns


def security_volatilities(data):
    volatilities = data.groupby('CUSIP')['RET'].std().reset_index()
    volatilities.columns = ['CUSIP', 'Volatility']
    return volatilities


def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    merged_data = pd.merge(ret_data, vol_data, on='CUSIP')
    correlation_value = merged_data['Average Return'].corr(merged_data['Volatility'])
    return correlation_value

#Test
cleaned_data = clean_data("monthlycrsp_mod6.csv")
avg_returns = avg_return(cleaned_data)
volatilities = security_volatilities(cleaned_data)
correlation_value = correlation(avg_returns, volatilities)
print(correlation_value)

0.4483709072705959
